In [1]:
import numpy as np
import os
import sys
import shutil
from keras.utils import np_utils, generic_utils
from utils import load_data, get_minst_model

Using Theano backend.
/Users/hongjian/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/Users/hongjian/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def load_train_data():
    """
    Load training data from .npy files.
    
    # normalization, not sure whether will work
    scale = np.max(data)
    data /= scale
    mean = np.std(data)
    data -= mean
    """
    X = np.load('train.npy')
    y = np.load('label.npy')

    X = X.astype(np.float32)
    X /= 255
    
    
    return X, y

def train():
    """
    Training using minst models.
    """
    
    nb_classes = 6 
    min_val_loss = sys.float_info.max
    nb_iter = 200
    epochs_per_iter = 2
    batch_size = 64
    best_accuracy = 0.0
    
    print('Loading and compiling models...')
    conv_model = get_minst_model('weights_conv_best.hdf5')

    print('Loading training data...')
    X_train, y_train = load_train_data()
    y_train = np_utils.to_categorical(y_train, nb_classes) 
    
    print('-'*50)
    print('Training...')
    print('-'*50)

    for i in range(nb_iter):
        print('-'*50)
        print('Iteration {0}/{1}'.format(i + 1, nb_iter))
        print('-'*50)

        print('Fitting  model...')
        hist_conv = conv_model.fit(X_train, y_train,shuffle = True, validation_split=0.2, 
                  nb_epoch=epochs_per_iter,batch_size=batch_size, 
                  verbose=1,show_accuracy=True)
        loss = hist_conv.history['loss'][-1]
        val_loss = hist_conv.history['val_loss'][-1]
        
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            conv_model.save_weights('weights_conv_best.hdf5', overwrite=True)

In [ ]:
train()

Loading and compiling models...
Loading training data...
--------------------------------------------------

In [ ]:
loss, accuracy = model.evaluate(train_dataset, train_labels, show_accuracy=True, verbose=1)
print('loss: ', loss)
print('accuracy: ', accuracy)

In [ ]:
model.predict_classes(test_data, batch_size=128, verbose=1)

In [ ]:
def get_toy_model():
    model = Sequential()


    model.add(Convolution2D(4, 5, 5, border_mode='valid',input_shape=( 3, 128, 128))) 
    model.add(Activation('tanh'))


    model.add(Convolution2D(8, 3, 3, border_mode='valid'))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(16, 3, 3, border_mode='valid')) 
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(32, 3, 3, border_mode='valid')) 
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(1024, init='normal'))
    model.add(Activation('tanh'))
    

    model.add(Dense(2, init='normal'))
    model.add(Activation('softmax'))


    sgd = SGD(l2=0.0,lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='Adam',class_mode="categorical")
    return model
def center_normalize(x):
    """
    Custom activation for online sample-wise center and std. normalization
    """
    return (x - K.mean(x)) / K.std(x)


def get_model():
    model = Sequential()
    model.add(Activation(activation=center_normalize, input_shape=(3, 128, 128)))

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='valid'))
    model.add(Activation('relu'))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(96, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(96, 3, 3, border_mode='valid'))
    model.add(Activation('relu'))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(128, 2, 2, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(128, 2, 2, border_mode='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, W_regularizer=l2(1e-3)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))

    sgd = SGD(l2=0.0,lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='adadelta',class_mode="categorical")
    return model